In [ ]:
import requests
import csv

def get_repository_contents(owner, repo):
    url = f'https://api.github.com/repos/{owner}/{repo}/contents'
    response = requests.get(url)
    return response.json()

def collect_datasets(contents, dataset_dirs, extensions):
    datasets = []
    for item in contents:
        if item['type'] == 'dir' and item['name'].lower() in dataset_dirs:
            subdir_contents = get_repository_contents(owner, repo, item['path'])
            datasets.extend(collect_datasets(subdir_contents, dataset_dirs, extensions))
        elif item['type'] == 'file' and item['name'].lower().endswith(tuple(extensions)):
            dataset = {
                'filename': item['name'],
                'url': item['url'],
                'size': item['size'],
                'created_at': item['created_at'],
                'updated_at': item['updated_at']
            }
            datasets.append(dataset)
    return datasets

# define the GitHub repository owner and name
owner = 'username'
repo = 'repository-name'

# define the directories that may contain datasets
dataset_dirs = ['data', 'dataset', 'datasets', 'csv', 'json', 'txt']

# define the file extensions to include
extensions = ['.csv', '.json', '.txt']

# get the contents of the repository
contents = get_repository_contents(owner, repo)

# collect all datasets
datasets = collect_datasets(contents, dataset_dirs, extensions)

# log the dataset metadata to a CSV file
with open('datasets.csv', mode='w') as file:
    writer = csv.DictWriter(file, fieldnames=['filename', 'url', 'size', 'created_at', 'updated_at'])
    writer.writeheader()
    for dataset in datasets:
        writer.writerow(dataset)
